In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus :
    try:
        # memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs", len(logical_gpus), 'Logical GPUs' )
    except RuntimeError as e:
        print(e)

1 Physical GPUs 1 Logical GPUs


In [2]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [4]:
from tensorflow.keras.optimizers import RMSprop
model.compile (loss='binary_crossentropy', optimizer= RMSprop(lr=0.001), metrics=['accuracy'])

In [5]:
import os

train_dir = f'{os.getcwd()}/train'
validation_dir = f'{os.getcwd()}/validation'

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir, # This is the source directory for training images
    target_size = (300, 300),
    batch_size = 8,
    class_mode = 'binary' # Since we use binary_crossentropy loss, we need binary labels
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300),
    batch_size = 8,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch = 128,
    epochs = 15,
    validation_data = validation_generator,
    validation_steps = 32,
    verbose = 1
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 128 steps, validate for 32 steps
Epoch 1/15
128/128 [==============================] - 31s 239ms/step - loss: 0.4159 - accuracy: 0.8106 - val_loss: 2.5546 - val_accuracy: 0.7734
Epoch 2/15
128/128 [==============================] - 25s 193ms/step - loss: 0.3461 - accuracy: 0.9392 - val_loss: 1.6153 - val_accuracy: 0.8281
Epoch 3/15
128/128 [==============================] - 25s 193ms/step - loss: 0.1216 - accuracy: 0.9647 - val_loss: 1.7486 - val_accuracy: 0.7930
Epoch 4/15
128/128 [==============================] - 25s 192ms/step - loss: 0.0980 - accuracy: 0.9764 - val_loss: 2.6978 - val_accuracy: 0.8594
Epoch 5/15
128/128 [==============================] - 25s 192ms/step - loss: 0.1257 - accuracy: 0.9784 - val_loss: 2.7424 - val_accuracy: 0.7227
Epoch 6/15
128/128 [==============================] - 25s 193ms/step - loss: 0.0697 - accuracy: 0.9863 - val_loss: 2.7928 - val_accuracy: 0.7891
Epoch 7/15
128/128 [=================

In [22]:
import numpy as np
from keras.preprocessing import image
import os

current_dir = os.getcwd()
image_name = 'image6.jpg'
path = f'{current_dir}/internet_images/{image_name}'
img = image.load_img(path, target_size= (300,300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if(classes[0]>0.5):
    print(path+" is a human")
else:
    print(path+" is a horse")


[0.]
C:\Users\ASUS\Desktop\deep learning code\Tensorflow in practice specialization\course 1 introduction to tensorflow\week4/internet_images/image6.jpg is a horse


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGTERM)